In [1]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import re
import pandas as pd
import time

In [2]:
headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}

In [3]:
def get_links(categoryID, page):
    url ='https://terms.naver.com/list.nhn?'
    
    params = {
        'cid':46702, 
        'categoryId':categoryID, # 한국화 46739, 동양화 46740, 서양화 46741
        'page':page
        }
    
    resp = requests.get(url,params=params,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    aList = soup.select('div.info_area > strong > a')
    
    return (each['href'] for each in aList)

In [4]:
def get_info(page_url):
    url = 'https://terms.naver.com/'+page_url
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    try :
        # 제목
        title = soup.select_one('div.headword_title > h2').getText()
    except Exception :
        title = ""    
        
    try :
        # 아티스트
        artist = soup.select_one('div.headword_title > p.desc').getText()
    except Exception :
        artist = ""   

    try :
        # 작품
        art = soup.select_one('#innerImage0')['data-src']
    except Exception :
        art = ""   


    try :
        # 텍스트
        text= soup.select_one('#size_ct > p').getText()
    except Exception :
        text = ""   
    
    
    # 종류, 기법
    
    art_type = ""
    tech = ""
    movement = ""

    tmp = soup.select('div.wr_tmp_profile > div > table > tbody > tr > th ')
    
    for num in range(len(tmp)) :
        try : 
            if tmp[num].getText() == '종류' :
                art_type = soup.select_one('div.wr_tmp_profile > div > table > tbody > tr:nth-of-type({node_num}) > td'.format(node_num = num+1)).getText()
            elif tmp[num].getText() == '기법' :
                tech = soup.select_one('div.wr_tmp_profile > div > table > tbody > tr:nth-of-type({node_num}) > td'.format(node_num = num+1)).getText()
            elif tmp[num].getText() == '사조' : # ~~ 주의, 주로 서양화에 있음
                movement = soup.select_one('div.wr_tmp_profile > div > table > tbody > tr:nth-of-type({node_num}) > td'.format(node_num = num+1)).getText()
        except Exception as e:
            print(e)
            continue


    return (title, artist, art_type, tech, movement, art, text)

## 한국화 _ 5891점 중 750점

In [6]:
# 한국화 URL을 얻는다.

ko_arts_url = []
for page in tqdm(range(1,51)) :
    ko_arts_url.extend(get_links(46739, page)) # 한국화 46739, 동양화 46740, 서양화 46741
ko_arts_url

100%|██████████| 50/50 [01:10<00:00,  1.41s/it]


['/entry.nhn?docId=1551628&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975213&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975309&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975248&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975221&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975222&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1552673&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1550823&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1563794&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975243&cid=46702&categoryId=46739',
 '/entry.nhn?docId=2028334&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1552018&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975649&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1550751&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1541961&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1567751&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975214&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1640511&cid=46702&categoryId=46739',


In [7]:
# 첫번째 URL로 테스트

url = 'https://terms.naver.com/'+ko_arts_url[0]
resp = requests.get(url,headers=headers)
soup = BeautifulSoup(resp.content,'lxml')
print(soup.select_one('div.headword_title > h2'))
print(soup.select_one('div.headword_title > p.desc'))

<h2 class="headword">압구정</h2>
<p class="desc">정선</p>


In [8]:
get_info(ko_arts_url[0])

('압구정',
 '정선',
 '수묵채색화',
 '견본채색',
 '',
 'https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3329_000_93%2F20141111221814820_Q2GWSKSQH.jpg%2F2014071012020641.jpg%3Ftype%3Dw690_fst%26wm%3DN%22&twidth=690&theight=448&opts=17',
 "지금 현대아파트, 한양아파트 등 고층 아파트들이 숲을 이루고 있는 강남구 압구정동 일대의 본 모습이다. 잠실 쪽에서 서북으로 흘러오던 한강 줄기가 꺾어져 서남으로 흘러가는데 그 물모룽이를 이루는 언덕 위에 높이 세워진 것이 압구정이다. 압구정에 올라서면 한양 서울을 둘러싸고 있는 명산들을 한눈에 조망할 수 있었다. 이 그림에서도 압구정동 일대와 강 건너 옥수동, 금호동 일대가 한눈에 다 보인다. 그 뒤로 보이는 짙은 초록빛 산은 남산이고, 멀리로는 삼각산 연봉들이 아스라이 펼쳐져 있다. 압구정 뒤로 보이는 원산은 청계산과 우면산일 것이다. 경치가 이토록 빼어난 곳이니 역대 권문세가들이 항상 이곳을 탐내어 별장을 짓고자 했다. 압구정을 처음 지은 사람은 한명회였다. 그는 수양대군의 심복이 되어 간교한 꾀로 김종서와 안평대군 등 조정대신과 왕자들을 죽이고 수양대군으로 하여금 어린 조카 단종으로부터 왕위를 빼앗게 한 장본인이다. 세조와 성종대를 거치며 최고의 권신으로 세상을 농락하던 한명회는 만년에 이곳에 별장을 짓고 명나라 문인 예겸에게 압구정, 즉 '갈매기와 친하게 지내는 정자'라는 뜻의 이름을 받는다. 이 그림에서도 제법 규모가 큰 정자가 언덕 위에 덩그렇게 지어져 있는데, 이 그림이 그려지던 때는 누가 압구정의 주인이었던지 확실치 않다. 압구정이 서있는 높은 언덕 아래 층층이 이어진 강변구릉 위로는 기와집과 초가집들이 마을을 이루듯 들어서 있다.이 중에는 당시 서울 대가집들의 별장이 상당수 섞여 있을 

In [26]:
ko_arts = []
for each_url in tqdm(ko_arts_url) :
    ko_arts.append(get_info(each_url))


100%|██████████| 750/750 [14:17<00:00,  1.14s/it]


In [27]:
ko_arts

[('압구정',
  '정선',
  '수묵채색화',
  '견본채색',
  '',
  'https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3329_000_93%2F20141111221814820_Q2GWSKSQH.jpg%2F2014071012020641.jpg%3Ftype%3Dw690_fst%26wm%3DN%22&twidth=690&theight=448&opts=17',
  "지금 현대아파트, 한양아파트 등 고층 아파트들이 숲을 이루고 있는 강남구 압구정동 일대의 본 모습이다. 잠실 쪽에서 서북으로 흘러오던 한강 줄기가 꺾어져 서남으로 흘러가는데 그 물모룽이를 이루는 언덕 위에 높이 세워진 것이 압구정이다. 압구정에 올라서면 한양 서울을 둘러싸고 있는 명산들을 한눈에 조망할 수 있었다. 이 그림에서도 압구정동 일대와 강 건너 옥수동, 금호동 일대가 한눈에 다 보인다. 그 뒤로 보이는 짙은 초록빛 산은 남산이고, 멀리로는 삼각산 연봉들이 아스라이 펼쳐져 있다. 압구정 뒤로 보이는 원산은 청계산과 우면산일 것이다. 경치가 이토록 빼어난 곳이니 역대 권문세가들이 항상 이곳을 탐내어 별장을 짓고자 했다. 압구정을 처음 지은 사람은 한명회였다. 그는 수양대군의 심복이 되어 간교한 꾀로 김종서와 안평대군 등 조정대신과 왕자들을 죽이고 수양대군으로 하여금 어린 조카 단종으로부터 왕위를 빼앗게 한 장본인이다. 세조와 성종대를 거치며 최고의 권신으로 세상을 농락하던 한명회는 만년에 이곳에 별장을 짓고 명나라 문인 예겸에게 압구정, 즉 '갈매기와 친하게 지내는 정자'라는 뜻의 이름을 받는다. 이 그림에서도 제법 규모가 큰 정자가 언덕 위에 덩그렇게 지어져 있는데, 이 그림이 그려지던 때는 누가 압구정의 주인이었던지 확실치 않다. 압구정이 서있는 높은 언덕 아래 층층이 이어진 강변구릉 위로는 기와집과 초가집들이 마을을 이루듯 들어서 있다.이 중에는 당시 서울 대가집들의 별장이 상당수

In [40]:
ko_data = pd.DataFrame(ko_arts,columns=['title','artist','art_type','tech','movement','image','text'])
ko_data.head(100)

,artist,title,art_type,tech,movement,image,text
0,압구정,정선,수묵채색화,견본채색,,https://dthumb-phinf.pstatic.net/?src=%22https...,"지금 현대아파트, 한양아파트 등 고층 아파트들이 숲을 이루고 있는 강남구 압구정동 ..."
1,단원풍속도첩 중 씨름,김홍도,수묵채색화,지본담채(紙本淡彩)(ink and slight color on paper),,https://dthumb-phinf.pstatic.net/?src=%22https...,단원(檀園)의 화재(畵才)는 정형산수나 인물화에서보다도 사경산수화(寫景山水畵)와 풍...
2,세한도,김정희,수묵화,지본묵화(紙本墨畵)(ink on paper),,https://dthumb-phinf.pstatic.net/?src=%22https...,김정희(金正喜)는 1786년(정조(正祖)10년)에 경주김씨(慶州金氏) 집안에서 태어...
3,초충도,신사임당,수묵채색화,지본설채(紙本設彩)(ink and color on paper),,https://dthumb-phinf.pstatic.net/?src=%22https...,율곡(栗谷) 이이(李珥)의 어머니 되는 사임당(思任堂) 신부인(申夫人)은 1504년...
4,", 《혜원 전신첩》",신윤복,수묵채색화,지본채색(紙本彩色),,https://dthumb-phinf.pstatic.net/?src=%22https...,눈썹달이 침침하게 내리 비치고 있는 야밤중에 등불을 비춰 든 선비 차림의 젊은이가 ...
5,", 《혜원 전신첩》",신윤복,수묵채색화,지본채색(紙本彩色),,https://dthumb-phinf.pstatic.net/?src=%22https...,이 그림은 단옷날 추천(鞦韆: 그네타기)놀이를 나온 한 떼의 여인네들이 시냇가에 그...
6,포도,신사임당,한국화,견본수묵(絹本水墨),,https://dthumb-phinf.pstatic.net/?src=%22https...,"신사임당(1504-1551)은 화조나 초충을 잘 그렸던 것으로 알려져 있지만, 실제..."
7,마상청앵,김홍도,수묵채색화,지본담채,,https://dthumb-phinf.pstatic.net/?src=%22https...,마상청앵(馬上聽鶯: 말 위에서 꾀꼬리 소리 듣다.)녹음방초 무성하고 온갖 꽃이 만발...
8,황묘농접,김홍도,한국화,지본채색,,https://dthumb-phinf.pstatic.net/?src=%22https...,"황묘농접(黃猫弄蝶: 노란 고양이가 나비를 놀리다)대지가 연록색풀로 가득 차고, 바위..."
9,몽유도원도,안견,수묵채색화,견본담채(絹本淡彩)(ink and slight color on silk),,https://dthumb-phinf.pstatic.net/?src=%22https...,안견(安堅)은 조선조 초기의 화단(畵壇)을 석권하였으며 한국 회화사(繪畫史)에 있어...


## 동양화 _ 1544점 중 750점

In [35]:
# 동양화 URL을 얻는다.

oriental_arts_url = []
for page in tqdm(range(1,51)) :
    oriental_arts_url.extend(get_links(46740, page)) # 한국화 46739, 동양화 46740, 서양화 46741
ko_arts_url

100%|██████████| 50/50 [01:10<00:00,  1.41s/it]


['/entry.nhn?docId=1551628&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975213&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975309&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975248&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975221&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975222&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1552673&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1550823&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1563794&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975243&cid=46702&categoryId=46739',
 '/entry.nhn?docId=2028334&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1552018&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975649&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1550751&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1541961&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1567751&cid=46702&categoryId=46739',
 '/entry.nhn?docId=975214&cid=46702&categoryId=46739',
 '/entry.nhn?docId=1640511&cid=46702&categoryId=46739',


In [36]:
# 첫번째 URL로 테스트

url = 'https://terms.naver.com/'+oriental_arts_url[0]
resp = requests.get(url,headers=headers)
soup = BeautifulSoup(resp.content,'lxml')
print(soup.select_one('div.headword_title > h2'))
print(soup.select_one('div.headword_title > p.desc'))

<h2 class="headword">민화 호작도</h2>
None


In [37]:
get_info(oriental_arts_url[0])

('민화 호작도',
 '',
 '수묵채색화',
 '종이에 수묵담채(Ink and color on paper)',
 '',
 'https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3329_000_58%2F20141007112731618_MSKHCVP6Q.jpg%2F114_11400.jpg%3Ftype%3Dw690_fst%26wm%3DN%22&twidth=265&theight=530&opts=17',
 '호작도란 서낭신의 사자인 까치가 호랑이에게 신탁(神託)을 전하는 모습을 그린 작품으로, 일반적인 호작도는 해학적인 표정과 단순한 동세를 지닌 호랑이를 그리곤 하는데 위 작품의 경우 당당하고 맹렬한 본연의 모습이 더 부각되어 있다. 크게 몸을 비틀어 소나무와 엉켜 나무밑둥을 물고 있는 형상의 S자형 구도로 호랑이의 용맹함이 더욱 돋보인다. 화폭에 나타나는 표현기법 또한 다양한데, 털을 세필로 섬세하게 표현한 것과는 대조적으로 좌우에 배치된 까치와 소나무를 농묵으로 힘 있게 그려 구성간의 관계를 더욱 조화롭게 해주고 있다. 길조의 상징인 까치와 영험한 동물인 호랑이 그림을 걸어둠으로서 고단한 민생의 근심을 덜고자 했던 선조들의 염원을 느낄 수 있는 작품이다. ')

In [38]:
oriental_arts = []
for each_url in tqdm(oriental_arts_url) :
    oriental_arts.append(get_info(each_url))

100%|██████████| 750/750 [08:57<00:00,  1.40it/s]


In [39]:
oriental_arts

[('민화 호작도',
  '',
  '수묵채색화',
  '종이에 수묵담채(Ink and color on paper)',
  '',
  'https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3329_000_58%2F20141007112731618_MSKHCVP6Q.jpg%2F114_11400.jpg%3Ftype%3Dw690_fst%26wm%3DN%22&twidth=265&theight=530&opts=17',
  '호작도란 서낭신의 사자인 까치가 호랑이에게 신탁(神託)을 전하는 모습을 그린 작품으로, 일반적인 호작도는 해학적인 표정과 단순한 동세를 지닌 호랑이를 그리곤 하는데 위 작품의 경우 당당하고 맹렬한 본연의 모습이 더 부각되어 있다. 크게 몸을 비틀어 소나무와 엉켜 나무밑둥을 물고 있는 형상의 S자형 구도로 호랑이의 용맹함이 더욱 돋보인다. 화폭에 나타나는 표현기법 또한 다양한데, 털을 세필로 섬세하게 표현한 것과는 대조적으로 좌우에 배치된 까치와 소나무를 농묵으로 힘 있게 그려 구성간의 관계를 더욱 조화롭게 해주고 있다. 길조의 상징인 까치와 영험한 동물인 호랑이 그림을 걸어둠으로서 고단한 민생의 근심을 덜고자 했던 선조들의 염원을 느낄 수 있는 작품이다. '),
 ('부춘산거도',
  '황공망',
  '중국 회화',
  '종이에 수묵',
  '',
  'https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3329_000_93%2F20141110134916041_8LBMQL4LZ.jpg%2F2011021819004391.jpg%3Ftype%3Dw690_fst%26wm%3DN%22&twidth=690&theight=483&opts=17',
  ''),
 ('조춘도',
  '곽희',
  '중국 회화',
  '비단에 수묵담채',
  '',
  'https://d

In [41]:
oriental_data = pd.DataFrame(oriental_arts,columns=['title','artist','art_type','tech','movement','image','text'])
oriental_data.head(100)

,artist,title,art_type,tech,movement,image,text
0,민화 호작도,,수묵채색화,종이에 수묵담채(Ink and color on paper),,https://dthumb-phinf.pstatic.net/?src=%22https...,호작도란 서낭신의 사자인 까치가 호랑이에게 신탁(神託)을 전하는 모습을 그린 작품으...
1,부춘산거도,황공망,중국 회화,종이에 수묵,,https://dthumb-phinf.pstatic.net/?src=%22https...,
2,조춘도,곽희,중국 회화,비단에 수묵담채,,https://dthumb-phinf.pstatic.net/?src=%22https...,
3,"수드라 (인도 4계급 중 하나, 하인)",프란츠 발타자르 솔빈스,인도/이슬람미술,,,https://dthumb-phinf.pstatic.net/?src=%22https...,
4,인간의 형상을 한 신화속 동물,,동아시아미술,"세밀화 (회화), 물감(miniature (peinture), gouache)",,https://dthumb-phinf.pstatic.net/?src=%22https...,
5,제사 음식 준비,,동아시아미술,"세밀화 (회화), 물감(miniature (peinture), gouache)",,https://dthumb-phinf.pstatic.net/?src=%22https...,
6,매,김득신,수묵채색화,비단에 수묵채색(Ink and color on silk),,https://dthumb-phinf.pstatic.net/?src=%22https...,곁가지 앙상한 고목가지 위로 발톱을 잔뜩 웅크린 매 한 마리가 도약 직전 몸을 앞으...
7,옹정 황제의 초상,,동아시아미술,비단 (회화)(soie (peinture)),,https://dthumb-phinf.pstatic.net/?src=%22https...,
8,인도 여자,,인도/이슬람미술,"종이 위 인화, 알부민과 염화나트륨 용액을 칠한 기법(épreuve sur papi...",,https://dthumb-phinf.pstatic.net/?src=%22https...,
9,노안도,조석진,수묵화,지본수묵(紙本水墨)(encre noire sur papier),,https://dthumb-phinf.pstatic.net/?src=%22https...,노안도(蘆雁圖)라는 그림의 형식은 조선 말기부터 근대에 이르기까지 모든 화가들이 즐...


## 서양화 _ 69409점 중 750점

In [42]:
# 서양화 URL을 얻는다.

western_arts_url = []
for page in tqdm(range(1,51)) :
    western_arts_url.extend(get_links(46741, page)) # 한국화 46739, 동양화 46740, 서양화 46741
western_arts_url

100%|██████████| 50/50 [01:07<00:00,  1.36s/it]


['/entry.nhn?docId=974744&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974738&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974760&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974741&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974606&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974814&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974737&cid=46702&categoryId=46753',
 '/entry.nhn?docId=973274&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974750&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974922&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974722&cid=46702&categoryId=46741',
 '/entry.nhn?docId=1563430&cid=46702&categoryId=46753',
 '/entry.nhn?docId=976240&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974777&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974784&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974935&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974725&cid=46702&categoryId=46753',
 '/entry.nhn?docId=974782&cid=46702&categoryId=46753',
 '/entry.

In [43]:
# 첫번째 URL로 테스트

url = 'https://terms.naver.com/'+western_arts_url[0]
resp = requests.get(url,headers=headers)
soup = BeautifulSoup(resp.content,'lxml')
print(soup.select_one('div.headword_title > h2'))
print(soup.select_one('div.headword_title > p.desc'))

<h2 class="headword">별이 빛나는 밤</h2>
<p class="desc">빈센트 반 고흐</p>


In [44]:
get_info(western_arts_url[0])

('별이 빛나는 밤',
 '빈센트 반 고흐',
 '유화',
 '캔버스에 유채(Oil on canvas)',
 '후기인상주의',
 'https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3329_000_2%2F20140914200644906_OX4FKNFM1.jpg%2FBAL_285769.jpg%3Ftype%3Dw690_fst_n%26wm%3DY%22&twidth=690&theight=530&opts=17',
 '빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐

In [45]:
western_arts = []
for each_url in tqdm(western_arts_url) :
    western_arts.append(get_info(each_url))

100%|██████████| 750/750 [16:31<00:00,  1.32s/it]


In [46]:
western_arts

[('별이 빛나는 밤',
  '빈센트 반 고흐',
  '유화',
  '캔버스에 유채(Oil on canvas)',
  '후기인상주의',
  'https://dthumb-phinf.pstatic.net/?src=%22https%3A%2F%2Fdbscthumb-phinf.pstatic.net%2F3329_000_2%2F20140914200644906_OX4FKNFM1.jpg%2FBAL_285769.jpg%3Ftype%3Dw690_fst_n%26wm%3DY%22&twidth=690&theight=530&opts=17',
  '빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지

In [47]:
western_data = pd.DataFrame(western_arts,columns=['title','artist','art_type','tech','movement','image','text'])
western_data.head(100)

,artist,title,art_type,tech,movement,image,text
0,별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 ...
1,아를르의 포룸 광장의 카페 테라스,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"산업 사회로 이행해 가던 19세기 말, 정신적 가치에 대한 열망은 예술가들이 좀 더..."
2,연인(키스),구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,꽃이 흩뿌려진 작은 초원 위에 서 있는 두 연인은 주변과 분리되어 그들을 마치 후광...
3,해바라기,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"1888년, 반 고흐(Vincent Van Gogh, 1853-1890)는 프랑스 ..."
4,아를의 별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Huile sur toile),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"빈센트 반 고흐가 주고 받았던 서신들을 살펴보면, 그가 얼마나 ‘밤의 효과’에 매료..."
5,에펠탑의 신랑신부,마르크 샤갈,유화,캔버스에 유채(Huile sur toile),,https://dthumb-phinf.pstatic.net/?src=%22https...,말년의 마르크 샤갈이 거주한 라콜린 저택 응접실의 벽난로 위로 그림 한 점이 걸려있...
6,아를의 반 고흐의 방,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,반 고흐는 《반 고흐의 방》이라는 이름으로 세 작품을 그렸다. 가장 오래된 버전은 ...
7,모나리자,레오나르도 다 빈치,유화,패널에 유채(Huile sur bois),르네상스,https://dthumb-phinf.pstatic.net/?src=%22https...,"15세기경, 패널에 유채로, 크기는 77x53cm이며 프랑스 파리의 루브르 박물관(..."
8,유디트,구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"살로메(Salome), 세멜레(Semele), 메데아(Medea), 그리고 유디트(..."
9,행복한 눈물,로이 릭텐스타인,유화,캔버스에 마그나펜(Magna on canvas),,https://dthumb-phinf.pstatic.net/?src=%22https...,앤디 워홀과 함께 미국의 팝아트를 대표하는 로이 릭텐스타인(Roy Lichtenst...


## to_CSV

In [51]:
ko_data.to_csv('./한국화750_네이버백과.csv')
oriental_data.to_csv('./동양화750_네이버백과.csv')
western_data.to_csv('./서양화750_네이버백과.csv')

---
## 서양화 졸라 많이 다운받기

In [6]:
# 서양화 URL을 얻는다.

western_arts_url = []
for page in tqdm(range(1,4601)) :
    western_arts_url.extend(get_links(46741, page)) # 한국화 46739, 동양화 46740, 서양화 46741

100%|██████████| 4600/4600 [1:17:25<00:00,  1.01s/it]


In [ ]:
western_arts = []
for each_url in tqdm(western_arts_url) :
    western_arts.append(get_info(each_url))

western_data = pd.DataFrame(western_arts,columns=['title','artist','art_type','tech','movement','image','text'])
western_data.to_csv('./서양화_네이버백과.csv')

 42%|████▏     | 28665/69000 [6:07:12<8:36:42,  1.30it/s] 